In [ ]:
from cortexlib.mouse import CortexlabMouse
from cortexlib.images import CortexlabImages
from cortexlib.simclr import PreTrainedSimCLRModel
from cortexlib.predictor import NeuralResponsePredictor
from cortexlib.utils.file import write_json_file
from cortexlib.utils.plotting import simclr_colours
from cortexlib.utils.random import set_global_seed
from cortexlib.utils.logging import Logger
import matplotlib.pyplot as plt
import pandas as pd

logger = Logger()
set_global_seed()

In [ ]:
mouse = CortexlabMouse()

logger.progress("Computing null distributions for all neurons...")
null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)

real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, neural_responses, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

logger.info(f"Neural responses shape: {neural_responses.shape}")

In [ ]:
images = CortexlabImages()

logger.progress("Loading and preprocessing images shown to mouse...")
image_dataset = images.load_images_shown_to_mouse(mouse.image_ids)
logger.success("Images processed")

In [ ]:
simclr = PreTrainedSimCLRModel()

logger.progress("SimCLR: extracting features from images shown to mouse")
simclr_feats, labels = simclr.extract_features(image_dataset)
logger.success("SimCLR features extracted")

for layer, feats in simclr_feats.items():
    logger.info(f"SimCLR {layer} feats shape: {tuple(feats.shape)}")

In [ ]:
results = []

logger.progress("Running regressions: SimCLR Feats => Neural Data")
for layer, feats in simclr_feats.items():
    logger.progress(f"SimCLR {layer}")

    predictor = NeuralResponsePredictor()
    r_squared = predictor.compute_r_squared(feats, neural_responses)
    fev = predictor.compute_fev(feats, neural_responses)
    
    results.append({
        'layer': layer,
        'test_r2': r_squared['test_r2'],
        'mean_fev': fev['mean_fev']
    })

logger.success(f"Regressions complete")

In [ ]:
write_json_file(results, f'./results/simclr_feats_to_neural_data.json')

In [ ]:
results_df = pd.DataFrame(results)
results_df.head(12)

In [ ]:
layers = [r['layer'] for r in results]
mean_fevs = [r['mean_fev'] for r in results]

colors = [simclr_colours[layer] for layer in layers]

plt.figure(figsize=(8, 5))
plt.bar(layers, mean_fevs, color=colors)
plt.ylabel("Mean FEV")
plt.xlabel("SimCLR Layer")
plt.title("Raw Neural Data: Mean FEV per Layer")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()